In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from main import LitVAE, AudioDataset
import torch
import torch.nn.functional as F
from torchaudio.transforms import GriffinLim
import numpy as np
from scipy.spatial import geometric_slerp
from IPython.display import Audio
import librosa

In [ ]:
from pathlib import Path

In [ ]:
def sample(n_dims=64):
    x = np.random.standard_normal(n_dims)
    x = x / np.sqrt(x.dot(x))
    return x

In [ ]:
gl = GriffinLim(n_fft=2048, hop_length=512, power=1.0, n_iter=128).cuda()

In [ ]:
def latest_checkpoint(v=None):
    base_path = Path('../lightning_logs/')
    
    if v is None:
        version = 0    
        for f in base_path.iterdir():
            v = int(f.name.split('_')[1])
            version = v if v > version else version
    else:
        version = v
    
    base_path = base_path / f'version_{version}/checkpoints'
    checkpoint = next(base_path.glob('*.ckpt'))
    
    return checkpoint

In [ ]:
try:
    del model
except:
    pass
model = LitVAE.load_from_checkpoint(latest_checkpoint())

In [ ]:
model.cuda()
model.eval()
pass

In [ ]:
zs = []
start = sample(64)
s = 8.
e = s
block = 128
for i in range(32):
    end = sample(64)
    zs.append(geometric_slerp(start, end, np.linspace(0, 1, block, endpoint=False)) * (np.linspace(s, e, block))[:, None])
    start = end
    s, e = e, s
zs = np.concatenate(zs, axis=0)

zs = torch.from_numpy(zs.astype('float32')).cuda()

In [ ]:
with torch.no_grad():
    x_hat = model.vae.decoder(zs)
    y_hats = x_hat * 1024

In [ ]:
s = y_hats.cpu().numpy()

# plt.matshow(s)
# plt.show()

zeros = torch.zeros(y_hats.shape[0], 1, device=y_hats.device)
sound = gl(torch.cat([zeros, y_hats], dim=1).T)
Audio(sound.cpu().numpy(), rate=44100, normalize=True)

In [ ]:
PATH = '/home/kureta/Music/cello/Cello Samples/BachMinu1-00000-.wav'
y, sr = librosa.load(PATH, mono=True, sr=44100)
s = np.abs(librosa.stft(y, n_fft=2048, hop_length=512)) / 1024
plt.matshow(s.T)
plt.show()

Audio(y, rate=44100, normalize=False)

In [ ]:
with torch.no_grad():
    x_hat, _, _, _ = model.vae(torch.from_numpy(s[1:]).T.cuda())
    y_hats = x_hat * 1024

In [ ]:
plt.matshow(y_hats.cpu().numpy())
plt.show()

zeros = torch.zeros(y_hats.shape[0], 1, device=y_hats.device)
sound = gl(torch.cat([zeros, y_hats], dim=1).T)
Audio(sound.cpu().numpy(), rate=44100, normalize=False)